In [ ]:
import sys
from pathlib import Path

AVES_ROOT = Path("../..") if not "google.colab" in sys.modules else Path("aves_git")

EOD_PATH = AVES_ROOT / "data" / "external" / "EOD_STGO"
EOD_PATH

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd

# esto configura la calidad de la imagen. dependerá de tu resolución. el valor por omisión es 80
mpl.rcParams["figure.dpi"] = 150
# esto depende de las fuentes que tengas instaladas en el sistema.
mpl.rcParams["font.family"] = "Fira Sans Extra Condensed"

In [ ]:
# Load
zones = (
    gpd.read_file(AVES_ROOT / "data" / "processed" / "scl_zonas_urbanas.json")
)
zones.head()

In [ ]:
from aves.data import eod

viajes = eod.read_trips(EOD_PATH).merge(eod.read_people(EOD_PATH))

viajes["PesoLaboral"] = (
    viajes["FactorLaboralNormal"] * viajes["Factor_LaboralNormal"]
)

viajes = viajes[pd.notnull(viajes["PesoLaboral"])]

viajes.columns

In [ ]:
zonas_diff = viajes[viajes['Proposito'] == 'Al trabajo'].groupby(['ZonaDestino'])['PesoLaboral'].sum().rename('atraccion').to_frame().join(viajes[viajes['Proposito'] == 'Al trabajo'].groupby(['ZonaOrigen'])['PesoLaboral'].sum().rename('generacion'), how='outer').fillna(0).assign(diferencia=lambda x: x['atraccion'] - x['generacion'])

zonas_diff

In [ ]:
zones_with_values = zones.join(zonas_diff, on='ID', how='inner').drop_duplicates(subset='ID')
zones_with_values

In [ ]:
zones_with_values.plot(column='diferencia', cmap='Reds', legend=True)

In [ ]:
from libpysal.weights import Queen, spatial_lag

w_queen = Queen.from_dataframe(zones_with_values)
w_queen.n

In [ ]:
sns.heatmap(w_queen.full()[0])

In [ ]:
w_queen.transform

In [ ]:
w_queen.transform = 'R'

In [ ]:
sns.heatmap(w_queen.full()[0])

Definición de Spatial Lag

$y_{li} = \sum_j w_{ij} y_j$

In [ ]:
zones_with_values['diferencia_lag'] = spatial_lag.lag_spatial(w_queen, zones_with_values['diferencia'])
zones_with_values[['diferencia_lag', 'diferencia']]

In [ ]:
zones_with_values.plot('diferencia_lag', cmap='Reds')

In [ ]:
zones_with_values['diferencia_std'] = zones_with_values['diferencia'] - zones_with_values['diferencia'].mean()
zones_with_values['diferencia_lag_std'] = spatial_lag.lag_spatial(w_queen, zones_with_values['diferencia_std'])

In [ ]:
ax = zones_with_values.plot(x='diferencia_std', y='diferencia_lag_std', kind='scatter')
ax.axvline(0, c="k", alpha=0.5)
ax.axhline(0, c="k", alpha=0.5)

In [ ]:
from scipy.stats import linregress

reg_result = linregress(zones_with_values['diferencia_std'], zones_with_values['diferencia_lag_std'])
# slope, intercept, rvalue, pvalue
reg_result[0], reg_result[1], reg_result[2], reg_result[3]

Moran's I

$I = \frac{n}{\sum_i \sum_j w_{ij}} \frac{\sum_i \sum_j w_{ij} z_i z_j}{\sum_i z_i^2}$

Donde:

* $n$ es la cantidad de muestras
* $w_{ij}$ es el valor de spatial lag de area $j$ para area $i$
* $z_i$ es $x_i - \bar{x_i}$

In [ ]:
from esda.moran import Moran

moran = Moran(zones_with_values['diferencia'], w_queen)


In [ ]:
moran.I, moran.p_sim

Local Moran's I

$I_i = \frac{n * z_i}{\sum_i z_i^2} \sum_j w_{ij} z_j$

De hecho:

$I = \sum_{i = 1}^n \frac{I_i}{n}$

In [ ]:
from esda.moran import Moran_Local

lisa = Moran_Local(zones_with_values['diferencia'], w_queen)

In [ ]:
ax = sns.kdeplot(lisa.Is)
# Add one small bar (rug) for each observation
# along horizontal axis
sns.rugplot(lisa.Is, ax=ax);

In [ ]:
from splot import esda as esdaplot

esdaplot.lisa_cluster(lisa, zones_with_values, p=1)

In [ ]:
esdaplot.lisa_cluster(lisa, zones_with_values)

In [ ]:
zones_with_values['lisa_group'] = (lisa.q * (lisa.p_sim < 0.05).astype(np.int))
zones_with_values['lisa_group'].value_counts()

In [ ]:
zones_with_values['label'] = zones_with_values['lisa_group'].map({
    0: "Non-Significant",
    1: "HH",
    2: "LH",
    3: "LL",
    4: "HL",
})

In [ ]:
zones_with_values.groupby(['NOM_COMUNA', 'label']).size().unstack(fill_value=0)